In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import opensense_data_downloader_and_transformer as oddt
from opensense_data_downloader_and_transformer import add_cml_attributes

# Derive small example dataset from the OpenMRG dataset from SMHI with large CML dataset
source: https://zenodo.org/record/6673751

In [2]:
oddt.download_andersson_2022_OpenMRG(local_path='data/andersson_2022_OpenMRG/', print_output=True)

File already exists at desired location data/andersson_2022_OpenMRG/OpenMRG.zip
Not downloading!


In [3]:
df_metadata = pd.read_csv(os.path.join('data/andersson_2022_Ods_resG/cml/cml_metadata.csv'), 
                                        index_col=0)
ds = xr.open_dataset('data/andersson_2022_OpenMRG/cml/cml.nc').sel(time=slice('2015-08-27T00', '2015-08-27T23:59:51'))

# Add metadata with naming convention as currently used in pycomlink example data file
for col_name, ds_var_name in [
    ('NearLatitude_DecDeg', 'site_0_lat'),
    ('NearLongitude_DecDeg', 'site_0_lon'),
    ('FarLatitude_DecDeg', 'site_1_lat'),
    ('FarLongitude_DecDeg', 'site_1_lon'),
    ('Frequency_GHz', 'frequency'),
    ('Polarization', 'polarization'),
    ('Length_km', 'length'),
]:
    ds.coords[ds_var_name] = (
        ('sublink'), 
        [df_metadata[df_metadata.Sublink==sublink_id][
            col_name].values[0] for sublink_id in list(ds.sublink.values)]
    )

ds.attrs['comment'] += '\nMetadata added with preliminary code from opensense_data_downloader.py'

# add standard attributes
ds = add_cml_attributes(ds)

In [4]:
df_metadata = pd.read_csv(os.path.join('data/andersson_2022_Ods_resG/cml/cml_metadata.csv'))

In [5]:
df_metadata

,Link,Direction,Sublink,NearLatitude_DecDeg,NearLongitude_DecDeg,FarLatitude_DecDeg,FarLongitude_DecDeg,Frequency_GHz,Polarization,Length_km,NearAntennaDiameter_m,FarAntennaDiameter_m
0,10001,A,195,57.70368,11.99507,57.69785,11.99110,28.2065,Vertical,0.69144,0.3,0.3
1,10001,B,194,57.69785,11.99110,57.70368,11.99507,29.2145,Vertical,0.69144,0.3,0.3
2,10002,A,253,57.72539,11.98181,57.72285,11.97265,38.5280,Vertical,0.61455,0.2,0.2
3,10002,B,254,57.72285,11.97265,57.72539,11.98181,37.2680,Vertical,0.61455,0.2,0.2
4,10003,A,209,57.69247,11.97496,57.69406,11.97951,38.5280,Vertical,0.32374,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...
723,10362,B,719,57.65908,12.13829,57.64649,12.11750,28.2345,Vertical,1.87269,0.3,0.3
724,10363,A,704,57.65543,12.03017,57.62855,11.96720,28.1785,Vertical,4.80579,0.3,0.6
725,10363,B,703,57.62855,11.96720,57.65543,12.03017,29.1865,Vertical,4.80579,0.6,0.3
726,10364,A,702,57.70923,12.00797,57.71181,11.98477,28.2485,Vertical,1.41213,0.3,0.3


In [6]:
df_metadata=df_metadata.set_index(['Direction','Link']).sort_values([('Sublink')], ascending=True)

In [7]:
df_metadata

Sublink  NearLatitude_DecDeg  NearLongitude_DecDeg  \
Direction Link                                                        
B         10088        1             57.71470              11.91276   
A         10088        2             57.72416              11.90373   
B         10289        3             57.78109              12.03528   
A         10289        4             57.77905              12.03998   
          10023        5             57.68946              11.97961   
...                  ...                  ...                   ...   
          10360      724             57.70588              11.94409   
          10353      725             57.70608              11.93871   
B         10353      726             57.70588              11.94409   
A         10352      727             57.70802              11.94824   
B         10352      728             57.70588              11.94409   

                 FarLatitude_DecDeg  FarLongitude_DecDeg  Frequency_GHz  \
Direction Link                                                            
B         10088            57.72416             11.90373        38.5840   
A         10088            57.71470             11.91276        37.3240   
B         10289            57.77905             12.03998        28.2065   
A         10289            57.78109             12.03528        29.2145   
          10023            57.69179             11.98384        38.5560   
...                             ...                  ...            ...   
          10360            57.72355             11.88980        32.3330   
          10353            57.70588             11.94409        38.5840   
B         10353            57.70608             11.93871        37.3240   
A         10352            57.70588             11.94409        38.5000   
B         10352            57.70802             11.94824        37.2400   

                Polarization  Length_km  NearAntennaDiameter_m  \
Direction Link                                                   
B         10088     Vertical    1.18227                    0.3   
A         10088     Vertical    1.18227                    0.3   
B         10289     Vertical    0.36079                    0.3   
A         10289     Vertical    0.36079                    0.3   
          10023     Vertical    0.36201                    0.3   
...                      ...        ...                    ...   
          10360     Vertical    3.78685                    0.3   
          10353     Vertical    0.32142                    0.3   
B         10353     Vertical    0.32142                    0.3   
A         10352     Vertical    0.34382                    0.3   
B         10352     Vertical    0.34382                    0.3   

                 FarAntennaDiameter_m  
Direction Link                         
B         10088                   0.3  
A         10088                   0.3  
B         10289                   0.3  
A         10289                   0.3  
          10023                   0.3  
...                               ...  
          10360                   0.3  
          10353                   0.3  
B         10353                   0.3  
A         10352                   0.3  
B         10352                   0.3  

[728 rows x 10 columns]

In [8]:
ds_multindex = ds.assign_coords({'sublink':df_metadata.index})

In [9]:
ds_multindex=ds_multindex.unstack()

In [10]:
ds_multindex=ds_multindex.rename({'Direction':'sublink_id','Link':'cml_id'})

In [11]:
ds_multindex['polarization'] = xr.where(ds_multindex['polarization'] == 'Vertical', 'v', 'h')

In [12]:
ds_multindex.tsl.attrs.pop('missing_value')
ds_multindex.tsl.attrs

ds_multindex.rsl.attrs.pop('missing_value')
ds_multindex.rsl.attrs

{'units': 'dBm',
 'coordinates': 'cml_id, sublink_id, time',
 'long_name': 'received_signal_level'}

In [13]:
ds_multindex.to_netcdf(
    path='openMRG_example.nc',
    encoding={
        'rsl': {
            'dtype': 'int32',
            'scale_factor': 0.01,
            '_FillValue': -9999,
            'zlib': True, 
            'complevel': 4},
        'tsl':{
            'dtype': 'int32',
            'scale_factor': 0.01,
            '_FillValue': -9999,
            'zlib': True, 
            'complevel': 4}
    }
)
